## The QUEST Dataset

This notebook covers the analysis of the "quest" dataset prepared for the implementation of this project. This project gathers the quest information from the following games to implement and evaluate the performance of popular LLMs (large language models) for the procedural generation of quests:

- Baldur's Gate[^1]
- Baldur's Gate II: Shadows of Amn[^2]
- The Elder Scrolls IV: Oblivion[^3]
- The Elder Scrolls V: Skyrim[^4]
- Minecraft[^5]
- Torchlight II[^6]
- Fallout[^7] †
- Fallout 2[^8] †

> _† Quest data for these games is collected manually with assistance of an LLM (to simplify the representations)._

**Sources:**

- S. Värtinen, P. Hämäläinen and C. Guckelsberger, "Generating Role-Playing Game Quests With GPT Language Models," in _IEEE Transactions on Games_, vol. 16, no. 1, pp. 127-139, March 2024, doi: 10.1109/TG.2022.3228480, Paper: https://ieeexplore.ieee.org/document/9980408, Dataset: https://osf.io/jtqdb/, Accessed On: 4 April 2025.
- Fallout Quests, _Nukapedia: The Fallout Wiki_, Available: https://fallout.fandom.com/wiki/Fallout_quests, Accessed On: 4 April, 2025.
- Fallout 2 Quests, _Nukapedia: The Fallout Wiki_, Available: https://fallout.fandom.com/wiki/Fallout_2_quests, Accessed On: 4 April 2025.

**Findings:**

- The dataset has some "rare" columns that will likely be empty, retained for further expressions of the quest.
- The relative distribution of quests for [^4] is the highest (34.3034%) and [^7] is the lowest (5.1146%).
- The quests added from the _Fallout_ games increase the total number of entries (Värtinen et al.) by 156.

<!-- References -->

[^1]: https://en.wikipedia.org/wiki/Baldur%27s_Gate_(video_game)
[^2]: https://en.wikipedia.org/wiki/Baldur%27s_Gate_II:_Shadows_of_Amn
[^3]: https://elderscrolls.fandom.com/wiki/The_Elder_Scrolls_IV:_Oblivion
[^4]: https://elderscrolls.fandom.com/wiki/The_Elder_Scrolls_V:_Skyrim
[^5]: https://www.minecraft.net/
[^6]: https://www.torchlight2.com/
[^7]: https://fallout.fandom.com/wiki/Fallout
[^8]: https://fallout.fandom.com/wiki/Fallout_2


In [ ]:
from pathlib import Path


# Map identifiers to game titles
id_to_title_map: dict[str, str] = {
    "BG": "Baldur's Gate",
    "BGII": "Baldur's Gate II: Shadows of Amn",
    "oblivion": "The Elder Scrolls IV: Oblivion",
    "skyrim": "The Elder Scrolls V: Skyrim",
    "MC": "Minecraft",
    "TL2": "Torchlight II",
    "fallout": "Fallout",
    "fallout2": "Fallout 2",
}

# Look through the dataset directory
# and collect paths to the JSON files
data_dir: Path = Path("../data/raw")
paths: list[Path] = []
for filepath in data_dir.rglob("*.json"):
    print(filepath)
    paths.append(filepath)

../data/raw/TL2/quests_TL2.json
../data/raw/TES/quests_skyrim.json
../data/raw/TES/quests_oblivion.json
../data/raw/MC/quests_MC.json
../data/raw/BG/quests_BG.json
../data/raw/BG/quests_BGII.json
../data/raw/FO/quests_fallout2.json
../data/raw/FO/quests_fallout.json


In [ ]:
# Map game titles to their file paths
title_to_path_map: dict[str, Path] = {}

for path in paths:
    filename: str = path.stem  # Extract filename without extension
    identifier: str = filename.split("_")[-1]  # Extract game identifier
    if identifier in id_to_title_map:
        title_to_path_map[id_to_title_map[identifier]] = path

title_to_path_map

{'Torchlight II': PosixPath('../data/raw/TL2/quests_TL2.json'),
 'The Elder Scrolls V: Skyrim': PosixPath('../data/raw/TES/quests_skyrim.json'),
 'The Elder Scrolls IV: Oblivion': PosixPath('../data/raw/TES/quests_oblivion.json'),
 'Minecraft': PosixPath('../data/raw/MC/quests_MC.json'),
 "Baldur's Gate": PosixPath('../data/raw/BG/quests_BG.json'),
 "Baldur's Gate II: Shadows of Amn": PosixPath('../data/raw/BG/quests_BGII.json'),
 'Fallout 2': PosixPath('../data/raw/FO/quests_fallout2.json'),
 'Fallout': PosixPath('../data/raw/FO/quests_fallout.json')}

In [ ]:
import pandas as pd


# Load and consolidate the datasets
quest_data_map: dict[str, pd.DataFrame] = {
    title: pd.read_json(path) for title, path in title_to_path_map.items()
}

for title, data in quest_data_map.items():
    data["title"] = title  # Associate the data with their respective titles

quest_data_map["Fallout"].head(10)

,name,objective,first_tasks,first_task_locations,quest_giver,reward,tools,description,title
0,Find the Water Chip,Locate and retrieve a replacement water chip f...,"[Exit Vault 13 and explore the wasteland., Tra...","[{'name': 'Vault 13', 'description': 'The play...","{'name': 'Vault 13 Overseer', 'description': '...","[{'name': 'Experience Points', 'description': ...",[Rope],"""Our vault's water chip has failed. Without it...",Fallout
1,Calm Rebel Faction,Convince the rebel faction in Vault 13 to rema...,[Speak to upset Vault 13 citizens to learn abo...,"[{'name': 'Vault 13 Living Quarters', 'descrip...","{'name': 'Upset Vault Dweller', 'description':...","[{'name': 'Experience Points', 'description': ...",[None],Upon returning to Vault 13 during the search f...,Fallout
2,Find the Water Thief,Identify and stop the person stealing water fr...,[Investigate reports of missing water rations ...,"[{'name': 'Vault 13 Storage Area', 'descriptio...","{'name': 'Vault 13 Overseer', 'description': '...","[{'name': 'Experience Points', 'description': ...",[],"Water rations in Vault 13 are disappearing, an...",Fallout
3,Destroy the Mutant Leader,"Eliminate the Master, leader of the mutant army.",[Infiltrate the Cathedral and locate the Master.],"[{'name': 'The Cathedral', 'description': 'A h...","{'name': 'Brotherhood of Steel Elders', 'descr...","[{'name': 'Experience Points', 'description': ...","[Power Armor, Plasma Rifle, Stealth Boy]","The Master, a mutated being seeking to unite a...",Fallout
4,Destroy the Source of the Mutants,Destroy the Military Base producing Super Muta...,[Locate the Mariposa Military Base and find a ...,"[{'name': 'Mariposa Military Base', 'descripti...","{'name': 'Brotherhood of Steel Elders', 'descr...","[{'name': 'Experience Points', 'description': ...","[C4 Explosives, Stealth Boy, Plasma Rifle]",The Vault Dweller must infiltrate the Mariposa...,Fallout
5,Obtain More Equipment,Acquire better equipment to increase survivabi...,[Find sources of better armor and weapons.],"[{'name': 'Gun Runners', 'description': 'A mer...","{'name': 'Self-initiated', 'description': 'The...","[{'name': 'Improved Gear', 'description': 'Acc...","[Caps (for purchasing equipment), Lockpick (fo...",As the Vault Dweller progresses through the wa...,Fallout
6,Research Important Information,Gather crucial intelligence to aid in survival...,[Locate sources of valuable information.],"[{'name': 'The Brotherhood of Steel', 'descrip...","{'name': 'Self-initiated', 'description': 'The...","[{'name': 'Crucial Intel', 'description': 'New...",[High Intelligence (for analyzing information)...,The Vault Dweller must seek out and analyze im...,Fallout
7,Unlock the Armory,Gain access to a restricted armory to acquire ...,[Find a way to bypass or unlock the armory doo...,"[{'name': 'Military Base', 'description': 'A h...","{'name': 'Self-initiated or Faction Leader', '...","[{'name': 'Advanced Weaponry', 'description': ...",[Lockpicking Skill (to manually bypass securit...,The Vault Dweller must find a way to enter a s...,Fallout
8,Rescue Tandi from the Raiders,"Rescue Tandi, the daughter of Aradesh, from th...",[Find Tandi's location within the Khans' camp.],"[{'name': 'Khan Raider Camp', 'description': '...","{'name': 'Aradesh', 'description': 'The leader...","[{'name': 'Experience Points', 'description': ...","[Speech (to negotiate with the Khans), Stealth...","Tandi, the daughter of Aradesh, has been kidna...",Fallout
9,Stop the Radscorpions,Eliminate the radscorpions threatening Shady S...,[Locate the radscorpion cave and eliminate the...,"[{'name': 'Radscorpion Cave', 'description': '...","{'name': 'Aradesh', 'description': 'The leader...","[{'name': 'Experience Points', 'description': ...",[Melee or ranged weapons to fight radscorpions...,The village of Shady Sands is plagued by deadl...,Fallout


In [ ]:
# Check the quest count for each game
print("Title of the Game".ljust(32), "|", "Quest Count")
print("-" * 32, "+", "-" * 11)
for title, data in quest_data_map.items():
    print(title.ljust(32), "|", str(len(data)).rjust(11))

Title of the Game                | Quest Count
-------------------------------- + -----------
Torchlight II                    |          80
The Elder Scrolls V: Skyrim      |         389
The Elder Scrolls IV: Oblivion   |         215
Minecraft                        |         100
Baldur's Gate                    |         100
Baldur's Gate II: Shadows of Amn |          94
Fallout 2                        |          98
Fallout                          |          58


In [5]:
# Combine into the quest dataset
quest_df: pd.DataFrame = pd.concat(quest_data_map, ignore_index=True)
quest_df.head(10)

,name,objective,first_tasks,first_task_locations,quest_giver,reward,characters,tools,description,locations,items,enemies,groups,title,motivation
0,Hidden Costs,bring enchanting tools to Baldrick the Enchanter,[search Path of the Honored Dead for a Beornen],"[{'name': 'Path of the Honored Dead', 'descrip...","{'name': 'Baldrick the Enchanter', 'descriptio...","[{'name': '', 'description': 'the player's fir...","[{'name': 'Beornen', 'description': 'a male be...",[Baldrick the Enchanter can't enchant anything...,"Welcome! Welcome, my friend. I have numerous e...",NaN,NaN,NaN,NaN,Torchlight II,NaN
1,Little Lost Ones,help Bellethe find Finnas and their precious a...,[find Finnas],"[{'name': 'Widow's Veil', 'description': 'a ca...","{'name': 'Bellethe', 'description': 'a worried...","[{'name': 'NONE', 'description': '', 'amount':...","[{'name': 'Finnas', 'description': 'Bellethe's...",[Bellethe and Finnas are homesteaders with a s...,"Please, help me! Our dear little ones wandered...","[{'name': 'Estherian Enclave', 'description': ...",NaN,NaN,NaN,Torchlight II,NaN
2,The Thing,destroy the Thing before it infects the whole ...,[find the Thing],"[{'name': 'Norsk Leiren', 'description': 'a co...","{'name': 'Blair', 'description': 'a researcher...","[{'name': 'NONE', 'description': '', 'amount':...","[{'name': 'Bennings', 'description': 'one of B...","[the Thing got in Bennings, but then Fuchs dis...","""Blair's Notes""\nby Blair\n\nIt was in the ice...",NaN,"[{'name': '""Blair's Notes""', 'description': 'n...","[{'name': 'Thing', 'description': 'a dangerous...",NaN,Torchlight II,NaN
3,Bounty: Sturmbeorn!,alleviate the threat of the Sturmbeorn,[kill 10 Sturmbeorn],"[{'name': 'Temple Steppes', 'description': 'a ...","{'name': 'Commander Vale', 'description': 'the...","[{'name': '', 'description': 'bounty', 'amount...",NaN,[NONE],BOUNTY!\nThe Sturmbeorn raiders are a threat t...,NaN,"[{'name': '""Bounty Board""', 'description': 'a ...","[{'name': 'Sturmbeorn', 'description': 'savage...","[{'name': 'Vanquisher Corps', 'description': '...",Torchlight II,NaN
4,Bounty: Warbeasts!,help colonists by thinning out packs of Warbeasts,[kill 10 Warbeasts],"[{'name': 'Temple Steppes', 'description': 'a ...","{'name': 'Commander Vale', 'description': 'the...","[{'name': 'NONE', 'description': '', 'amount':...",NaN,[NONE],BOUNTY!\nCommander Vale of the Vanquisher Corp...,"[{'name': 'Estheria', 'description': 'a countr...","[{'name': '""Bounty Board""', 'description': 'a ...","[{'name': 'Warbeasts', 'description': 'dangero...","[{'name': 'Vanquisher Corps', 'description': '...",Torchlight II,NaN
5,Cacklespit's Brew,collect ingredients for Cacklespit's brew,"[get Mask of a Murderer, Thorns as Hard as Iro...","[{'name': 'Blightbogs', 'description': 'a bog'}]","{'name': 'Cacklespit', 'description': 'a potio...","[{'name': '', 'description': 'a special concoc...",NaN,[NONE],Another customer for one of old Cacklespit's b...,NaN,"[{'name': 'Mask of a Murderer', 'description':...",NaN,NaN,Torchlight II,NaN
6,Heartfire,recover the Heartfire Stone for yourself,[claim the key to Vyrax's Tower],"[{'name': 'Forgotten Halls', 'description': 'a...","{'name': 'Cam', 'description': 'a researcher t...","[{'name': 'Heartfire Stone', 'description': 'a...","[{'name': 'Vyrax', 'description': 'an ancient ...",[NONE],Hello again! I think I may be able to repay th...,"[{'name': '', 'description': 'Vyrax's Tower, a...",NaN,NaN,NaN,Torchlight II,NaN
7,Truffle Kerfuffle,bring 12 Ember Truffles to Clive,[gather 12 Ember Truffles],"[{'name': 'Muckery', 'description': 'a fungal ...","{'name': 'Clive', 'description': 'a botanist w...","[{'name': 'Super Mana Potion', 'description': ...",NaN,[NONE],"I'm a botanist, serving the Empire by gatherin...",NaN,"[{'name': 'Ember Truffles', 'description': 'Em...",NaN,NaN,Torchlight II,NaN
8,Protect the Guardian!,protect the Guardian of Water,[scout the Wellspring Temple],"[{'name': 'Wellspring Temple', 'description': ...","{'name': 'Eldrayn', '

In [6]:
quest_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1134 entries, 0 to 1133
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   name                  1134 non-null   object
 1   objective             1134 non-null   object
 2   first_tasks           1134 non-null   object
 3   first_task_locations  1134 non-null   object
 4   quest_giver           1134 non-null   object
 5   reward                1134 non-null   object
 6   characters            583 non-null    object
 7   tools                 1132 non-null   object
 8   description           1134 non-null   object
 9   locations             375 non-null    object
 10  items                 270 non-null    object
 11  enemies               25 non-null     object
 12  groups                372 non-null    object
 13  title                 1134 non-null   object
 14  motivation            100 non-null    object
dtypes: object(15)
memory usage: 133.0+ KB


In [ ]:
quest_df["title"].value_counts(normalize=True)

title
The Elder Scrolls V: Skyrim         0.343034
The Elder Scrolls IV: Oblivion      0.189594
Minecraft                           0.088183
Baldur's Gate                       0.088183
Fallout 2                           0.086420
Baldur's Gate II: Shadows of Amn    0.082892
Torchlight II                       0.070547
Fallout                             0.051146
Name: proportion, dtype: float64

In [8]:
quest_df.isnull().sum()  # Count of the missing values

name                       0
objective                  0
first_tasks                0
first_task_locations       0
quest_giver                0
reward                     0
characters               551
tools                      2
description                0
locations                759
items                    864
enemies                 1109
groups                   762
title                      0
motivation              1034
dtype: int64

In [ ]:
quest_df[
    (quest_df["tools"].astype(str) == "[]")
    | (quest_df["tools"].astype(str) == "['NONE']")
]  # Rows with empty tools list

,name,objective,first_tasks,first_task_locations,quest_giver,reward,characters,tools,description,locations,items,enemies,groups,title,motivation
3,Bounty: Sturmbeorn!,alleviate the threat of the Sturmbeorn,[kill 10 Sturmbeorn],"[{'name': 'Temple Steppes', 'description': 'a ...","{'name': 'Commander Vale', 'description': 'the...","[{'name': '', 'description': 'bounty', 'amount...",NaN,[NONE],BOUNTY!\nThe Sturmbeorn raiders are a threat t...,NaN,"[{'name': '""Bounty Board""', 'description': 'a ...","[{'name': 'Sturmbeorn', 'description': 'savage...","[{'name': 'Vanquisher Corps', 'description': '...",Torchlight II,NaN
4,Bounty: Warbeasts!,help colonists by thinning out packs of Warbeasts,[kill 10 Warbeasts],"[{'name': 'Temple Steppes', 'description': 'a ...","{'name': 'Commander Vale', 'description': 'the...","[{'name': 'NONE', 'description': '', 'amount':...",NaN,[NONE],BOUNTY!\nCommander Vale of the Vanquisher Corp...,"[{'name': 'Estheria', 'description': 'a countr...","[{'name': '""Bounty Board""', 'description': 'a ...","[{'name': 'Warbeasts', 'description': 'dangero...","[{'name': 'Vanquisher Corps', 'description': '...",Torchlight II,NaN
5,Cacklespit's Brew,collect ingredients for Cacklespit's brew,"[get Mask of a Murderer, Thorns as Hard as Iro...","[{'name': 'Blightbogs', 'description': 'a bog'}]","{'name': 'Cacklespit', 'description': 'a potio...","[{'name': '', 'description': 'a special concoc...",NaN,[NONE],Another customer for one of old Cacklespit's b...,NaN,"[{'name': 'Mask of a Murderer', 'description':...",NaN,NaN,Torchlight II,NaN
6,Heartfire,recover the Heartfire Stone for yourself,[claim the key to Vyrax's Tower],"[{'name': 'Forgotten Halls', 'description': 'a...","{'name': 'Cam', 'description': 'a researcher t...","[{'name': 'Heartfire Stone', 'description': 'a...","[{'name': 'Vyrax', 'description': 'an ancient ...",[NONE],Hello again! I think I may be able to repay th...,"[{'name': '', 'description': 'Vyrax's Tower, a...",NaN,NaN,NaN,Torchlight II,NaN
7,Truffle Kerfuffle,bring 12 Ember Truffles to Clive,[gather 12 Ember Truffles],"[{'name': 'Muckery', 'description': 'a fungal ...","{'name': 'Clive', 'description': 'a botanist w...","[{'name': 'Super Mana Potion', 'description': ...",NaN,[NONE],"I'm a botanist, serving the Empire by gatherin...",NaN,"[{'name': 'Ember Truffles', 'description': 'Em...",NaN,NaN,Torchlight II,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1088,Recruit Ian,Convince Ian to join as a companion.,[Talk to Ian and persuade him to join.],"[{'name': 'Shady Sands', 'description': 'A sma...","{'name': 'Ian', 'description': 'A former carav...","[{'name': 'Ian's Assistance', 'description': '...",NaN,[],"Ian, a former caravan guard, is willing to joi...",NaN,NaN,NaN,NaN,Fallout,NaN
1091,Join the Khans,Prove loyalty to the Khans and become a member.,"[Speak with Garl, the leader of the Khans., Co...","[{'name': 'Khans' Camp', 'description': 'A rai...","{'name': 'Garl', 'description': 'Leader of the...","[{'name': 'Khans Membership', 'description': '...",NaN,[],The player must speak to Garl and prove their ...,NaN,NaN,NaN,NaN,Fallout,NaN
1126,Receive Your Weekly Issue,Collect the latest edition of the weekly publi...,[Find the newspaper vendor or designated distr...,"[{'name': 'Hub Marketplace', 'description': 'A...","{'name': 'Newspaper Vendor', 'description': 'A...","[{'name': 'Weekly Issue', 'description': 'A pu...",NaN,[],A routine task to collect the latest issue of ...,NaN,NaN,NaN,NaN,Fallout,NaN
1127,Dr. Lorri's Surgeries,Undergo surgery to enhance physical attributes.,"[Find Dr. Lorri at the medical facility., Cons...","[{'name': 'Medical Facility', 'description': '...","{'name': 'Dr. Lorri', 'description': 'A skille...","[{'name': 'Physical Enhancement', 'description...",NaN,[],Dr. Lorri offers surgical procedures that enha...,NaN,NaN,NaN,NaN,Fallout,NaN


In [ ]:
# Fill rows for empty tools ([], null) with ['NONE']
quest_df.loc[
    (quest_df["tools"].astype(str) == "[]") | (quest_df["tools"].isnull()), "tools"
] = [["NONE"]]

In [ ]:
from sklearn.model_selection import train_test_split


train_df: pd.DataFrame
val_df: pd.DataFrame
test_df: pd.DataFrame

# Stratify split the quest dataset
train_df, test_df = train_test_split(
    quest_df, test_size=0.2, random_state=42, stratify=quest_df["title"]
)
val_df, test_df = train_test_split(
    test_df, test_size=0.5, random_state=42, stratify=test_df["title"]
)

len(train_df), len(val_df), len(test_df)

(907, 113, 114)

In [12]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 907 entries, 69 to 298
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   name                  907 non-null    object
 1   objective             907 non-null    object
 2   first_tasks           907 non-null    object
 3   first_task_locations  907 non-null    object
 4   quest_giver           907 non-null    object
 5   reward                907 non-null    object
 6   characters            474 non-null    object
 7   tools                 907 non-null    object
 8   description           907 non-null    object
 9   locations             297 non-null    object
 10  items                 208 non-null    object
 11  enemies               18 non-null     object
 12  groups                302 non-null    object
 13  title                 907 non-null    object
 14  motivation            80 non-null     object
dtypes: object(15)
memory usage: 113.4+ KB


In [ ]:
train_df["title"].value_counts(normalize=True)

title
The Elder Scrolls V: Skyrim         0.342889
The Elder Scrolls IV: Oblivion      0.189636
Baldur's Gate                       0.088203
Minecraft                           0.088203
Fallout 2                           0.085998
Baldur's Gate II: Shadows of Amn    0.082690
Torchlight II                       0.070562
Fallout                             0.051819
Name: proportion, dtype: float64

In [14]:
val_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 113 entries, 980 to 716
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   name                  113 non-null    object
 1   objective             113 non-null    object
 2   first_tasks           113 non-null    object
 3   first_task_locations  113 non-null    object
 4   quest_giver           113 non-null    object
 5   reward                113 non-null    object
 6   characters            50 non-null     object
 7   tools                 113 non-null    object
 8   description           113 non-null    object
 9   locations             38 non-null     object
 10  items                 32 non-null     object
 11  enemies               2 non-null      object
 12  groups                33 non-null     object
 13  title                 113 non-null    object
 14  motivation            10 non-null     object
dtypes: object(15)
memory usage: 14.1+ KB


In [ ]:
val_df["title"].value_counts(normalize=True)

title
The Elder Scrolls V: Skyrim         0.345133
The Elder Scrolls IV: Oblivion      0.185841
Fallout 2                           0.088496
Baldur's Gate                       0.088496
Minecraft                           0.088496
Baldur's Gate II: Shadows of Amn    0.079646
Torchlight II                       0.070796
Fallout                             0.053097
Name: proportion, dtype: float64

In [16]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 114 entries, 1019 to 537
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   name                  114 non-null    object
 1   objective             114 non-null    object
 2   first_tasks           114 non-null    object
 3   first_task_locations  114 non-null    object
 4   quest_giver           114 non-null    object
 5   reward                114 non-null    object
 6   characters            59 non-null     object
 7   tools                 114 non-null    object
 8   description           114 non-null    object
 9   locations             40 non-null     object
 10  items                 30 non-null     object
 11  enemies               5 non-null      object
 12  groups                37 non-null     object
 13  title                 114 non-null    object
 14  motivation            10 non-null     object
dtypes: object(15)
memory usage: 14.2+ KB


In [ ]:
test_df["title"].value_counts(normalize=True)

title
The Elder Scrolls V: Skyrim         0.342105
The Elder Scrolls IV: Oblivion      0.192982
Fallout 2                           0.087719
Minecraft                           0.087719
Baldur's Gate                       0.087719
Baldur's Gate II: Shadows of Amn    0.087719
Torchlight II                       0.070175
Fallout                             0.043860
Name: proportion, dtype: float64

In [ ]:
# Save the split datasets for further use
train_df.to_json("../data/quests_train.json", orient="records", indent=2)
val_df.to_json("../data/quests_val.json", orient="records", indent=2)
test_df.to_json("../data/quests_test.json", orient="records", indent=2)